#### LCEL (LangChain Expression Language) ແມ່ນຫຍັງ:
**LCEL ແມ່ນ syntax ຂອງ LangChain ທີ່ໃຊ້ໃນການເຊື່ອມຕໍ່ components ຕ່າງໆເຂົ້າດ້ວຍກັນ ແລະ ສາມາດເຮັດໃຫ້ອ່ານງ່າຍ. ໂດຍມັກຈະໃຊ້ເຄື່ອງໝາຍ | (pipe) ເພື່ອເຊື່ອມຕໍ່ components ຕ່າງໆໃນລຳດັບ.**

**HumanMessage:**
- ເປັນ class ສຳຫຼັບຈັດເກັບຂໍ້ຄວາມຂອງຜູ້ໃຊ້
- ແທນຄຳສັ່ງ ຫຼື ຄຳຖາມທີ່ User ສົ່ງມາ 

**SystemMessage:**
- ເປັນ class ສຳຫຼັບຈັດເກັບຄຳສັ່ງຂອງລະບົບ
- ກຳນົດບົດບາດ ແລະ ພຶດຕິກຳຂອງ AI
- ຕັ້ງກົດລະບຽບໃຫ້ AI ປະຕິບັດຕາມ

**StrOutputParser:**
- ເປັນ parser ທີ່ປ່ຽນຜົນລັບຈາກ AI model ໃຫ້ເປັນ string
- ເອົາແຕ່ເນື້ອໃນຂອງການຕອບກັບ
- ກຳຈັດ metadata ແລະ ຂໍ້ມູນພິເສດອື່ນໆ

**Chain:**
- ເປັນການເຊື່ອມຕໍ່ components ຫຼາຍໂຕເຂົ້າດ້ວຍກັນ
- ສາມາດເຮັດວຽກແບບລຳດັບຂັ້ນຕອນ
- ຜົນລັບຈາກ component ກ່ອນຈະເປັນ input ຂອງ component ຕໍ່ໄປ

**ChatPromptTemplate:**
- ເປັນ template ສຳຫຼັບສ້າງ prompts ສຳຫຼັບ chat models
- ສາມາດປະກອບຫຼາຍ message types ເຂົ້າດ້ວຍກັນ
- ມີ variables ທີ່ສາມາດປ່ຽນແປງຄ່າໄດ້
- ຊ່ວຍໃຫ້ການສ້າງ prompts ເປັນລະບົບ ແລະ ນຳໃຊ້ຄືນໄດ້ 

### Optional
```python
import os
from dotenv import load_dotenv
load_dotenv() 
groq_api_key=os.getenv("GROQ_API_KEY")

from langchain_groq import ChatGroq
model=ChatGroq(model="meta-llama/llama-4-maverick-17b-128e-instruct",groq_api_key=groq_api_key)
model
```

In [ ]:
import os
from google.oauth2 import service_account
from langchain_google_vertexai import ChatVertexAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Configuration
JSON_KEY_PATH = r"C:\Users\Kenshin\Desktop\GoogleI-O\key.json" 
PROJECT_ID = os.getenv("GOOGLE_CLOUD_PROJECT") 
GOOGLE_LOCATION= os.getenv("GOOGLE_LOCATION")

try:
    # Load credentials
    credentials = service_account.Credentials.from_service_account_file(
        JSON_KEY_PATH,
        scopes=["https://www.googleapis.com/auth/cloud-platform"]
    )
    
    # Initialize Vertex AI model
    model = ChatVertexAI(
        model_name="gemini-2.5-flash-lite",  
        project=PROJECT_ID,
        location=GOOGLE_LOCATION, 
        credentials=credentials,
        temperature=1.0
    )
    
    # Test the model
    response = model.invoke("Hello! How are you today?")
    print(f"🤖 Response: {response.content}")

except Exception as e:
    print(f"Error occurred: {str(e)}")

🤖 Response: Hello! I'm doing well, thank you for asking. As a large language model, I don't experience feelings in the same way humans do, but I'm functioning optimally and ready to assist you.

How are **you** doing today? Is there anything I can help you with?


In [32]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are a professional English to Lao translator. Translate the following English text to Lao language accurately, maintaining the original meaning and tone. Provide only the Lao translation without explanations."),
    HumanMessage(content="Hello, How is going today?")
]

result = model.invoke(messages)

In [33]:
result

AIMessage(content='ສະບາຍດີ, ມື້ນີ້ເປັນແນວໃດ?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 56, 'total_tokens': 77, 'completion_time': 0.099357294, 'prompt_time': 0.0013883, 'queue_time': 0.057221749999999995, 'total_time': 0.100745594}, 'model_name': 'meta-llama/llama-4-maverick-17b-128e-instruct', 'system_fingerprint': 'fp_c527aa4474', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--fdedd11e-57e1-4787-81e2-008d8b59c1db-0', usage_metadata={'input_tokens': 56, 'output_tokens': 21, 'total_tokens': 77})

In [34]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'ສະບາຍດີ, ມື້ນີ້ເປັນແນວໃດ?'

In [35]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'ສະບາຍດີ, ມື້ນີ້ເປັນແນວໃດ?'

In [40]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="You are a professional English to Lao translator. Translate the following English text to Lao language accurately, maintaining the original meaning and tone. Provide only the Lao translation without explanations. Translate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)

In [41]:
result=prompt.invoke({"language":"Lao","text":"Hello, what is Google I/O?"})

In [42]:
result.to_messages()

[SystemMessage(content='You are a professional English to Lao translator. Translate the following English text to Lao language accurately, maintaining the original meaning and tone. Provide only the Lao translation without explanations. Translate the following into Lao:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello, what is Google I/O?', additional_kwargs={}, response_metadata={})]

In [43]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"Lao","text":"Hello, what is Google I/O?"})

'ສະບາຍດີ, Google I/O ແມ່ນຫຍັງ?'